In [1]:
import requests
import pandas as pd

In [ ]:
api_key = 12345

In [2]:
def get_coordinates(location):
    api_key = api_key  # Replace with your actual API key
    base_url = 'https://maps.googleapis.com/maps/api/geocode/json'

    params = {
        'address': location + ', Canada',
        'key': api_key
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    if response.status_code == 200 and data['status'] == 'OK':
        result = data['results'][0]
        lat = result['geometry']['location']['lat']
        lng = result['geometry']['location']['lng']
        full_name = result['address_components'][0]['long_name']
        types = result['types']
        place_id = result['place_id']
        return lat, lng, types, full_name, place_id
    else:
        #print('Error occurred while geocoding:', data['status']," ", location)
        return 0,0,[],"",""

In [3]:
def generate_google_maps_link(place_id):
    base_url = "https://www.google.com/maps/place/?q=place_id:"
    return base_url + place_id

In [4]:
park_name =  'Banff National Park' #'Alexander Graham Bell Museum'

#park_name = park_name.lower().replace('national','provincial')
#print(park_name)

lat,lon,types,fname,place_id = get_coordinates(park_name)
place_id


'ChIJlZGSjCtmd1MR5tfKrGjincA'

In [5]:
file_path = 'entity_file.csv'

df = pd.read_csv(file_path)
df.head()

,Unnamed: 0,word,freq_count
0,477,Fundy Park,1
1,690,Moncton,1
2,688,Wood Buffalo,1
3,680,Ireland,1
4,670,Midland National Park,1


In [6]:
latitudes = []
longitudes = []
isparks = []
names = []
urls = []

In [7]:
for row in df.iterrows():
  lat,lon,types, full_name,plid = get_coordinates(row[1].word)
  if 'park' not in types:
    name = row[1].word.lower()
    name = name.replace('national','')
    name = name.replace('provincial','')
    name = name.replace('provintial','')
    name = name.replace('park','')
    name = name.replace('  ',' ')

    lat,lon,types,full_name,plid = get_coordinates(name + " national park")
    if ('park' in types) : print(row[1].word, " add npark")

    if 'park' not in types:
      lat,lon,types,full_name,plid = get_coordinates(name + " provincial park")
      if ('park' in types) : print(row[1].word, " add ppark")

  latitudes.append(lat)
  longitudes.append(lon)
  isparks.append('park' in types)
  names.append(full_name)
  urls.append(generate_google_maps_link(plid))

df['Lat'] = latitudes
df['Lon'] = longitudes
df['Ispark'] = isparks
df['Full_name'] = names
df['URL'] = urls

Fundy Park  add npark
Wood Buffalo  add npark
Midland National Park  add ppark
Nairn Falls  add npark
Joffre Lakes  add ppark
Coquihalla  add ppark
Kilarney  add ppark
Morne  add npark
Victoria  add ppark
Chignecto  add ppark
Fundy NP  add npark
Imo Banff  add npark
Georgian  add npark
Pingualuit Park  add npark
North  add ppark
Fathom  add npark
Nahanni  add npark
Stone  add ppark
Peter  add ppark
Bruce  add npark
Prince of Wales  add npark
Nahann  add npark
Parc National de la Mauricie  add npark
Le Bic  add npark
Rouge National Urban Park  add npark
Indian  add ppark
Riding Mountain National  add npark
Peter Lougheed  add ppark
Mount Assiniboine  add ppark
Mt Robson  add ppark
Kananaskis  add ppark
Sleeping Giant  add ppark
Kananask  add ppark
South Okanagan  add ppark
Cypress Hills  add ppark
Pigeon Park  add ppark
Gulf Islands  add npark
Robson  add ppark
Lakes  add npark
Cartier  add npark
Kouchiboug  add npark
Fathom Five National Marine Park  add npark
Stanhope  add npark
Minga

In [8]:
df.head()

,Unnamed: 0,word,freq_count,Lat,Lon,Ispark,Full_name,URL
0,477,Fundy Park,1,45.613777,-65.031631,True,Fundy National Park Of Canada,https://www.google.com/maps/place/?q=place_id:...
1,690,Moncton,1,46.087817,-64.778231,False,Moncton,https://www.google.com/maps/place/?q=place_id:...
2,688,Wood Buffalo,1,59.439503,-112.876402,True,Wood Buffalo National Park of Canada,https://www.google.com/maps/place/?q=place_id:...
3,680,Ireland,1,56.130366,-106.346771,False,Canada,https://www.google.com/maps/place/?q=place_id:...
4,670,Midland National Park,1,51.481096,-112.769190,True,Midland Provincial Park,https://www.google.com/maps/place/?q=place_id:...


In [9]:
df_fn = df.drop(columns = ['word','Unnamed: 0'])

In [10]:
df_fn['freq_count2'] = df_fn.groupby('Full_name')['freq_count'].transform('sum')

In [11]:
df_fn

,freq_count,Lat,Lon,Ispark,Full_name,URL,freq_count2
0,1,45.613777,-65.031631,True,Fundy National Park Of Canada,https://www.google.com/maps/place/?q=place_id:...,14
1,1,46.087817,-64.778231,False,Moncton,https://www.google.com/maps/place/?q=place_id:...,1
2,1,59.439503,-112.876402,True,Wood Buffalo National Park of Canada,https://www.google.com/maps/place/?q=place_id:...,1
3,1,56.130366,-106.346771,False,Canada,https://www.google.com/maps/place/?q=place_id:...,267
4,1,51.481096,-112.769190,True,Midland Provincial Park,https://www.google.com/maps/place/?q=place_id:...,2
...,...,...,...,...,...,...,...
465,27,51.253775,-85.323214,False,Ontario,https://www.google.com/maps/place/?q=place_id:...,31
466,31,53.933271,-116.576504,False,Alberta,https://www.google.com/maps/place/?q=place_id:...,33
467,35,52.873383,-117.954294,True,Jasper National Park Of Canada,https://www.google.com/maps/place/?q=place_id:...,38
468,35,56.130366,-106.346771,False,Canada,https://www.google.com/maps/place/?q=place_id:...,267


In [12]:
df_fn = df_fn.drop(columns = ['freq_count'])

In [13]:
df_fn = df_fn.drop_duplicates(keep='first')

In [14]:
df_fn

,Lat,Lon,Ispark,Full_name,URL,freq_count2
0,45.613777,-65.031631,True,Fundy National Park Of Canada,https://www.google.com/maps/place/?q=place_id:...,14
1,46.087817,-64.778231,False,Moncton,https://www.google.com/maps/place/?q=place_id:...,1
2,59.439503,-112.876402,True,Wood Buffalo National Park of Canada,https://www.google.com/maps/place/?q=place_id:...,1
3,56.130366,-106.346771,False,Canada,https://www.google.com/maps/place/?q=place_id:...,267
4,51.481096,-112.769190,True,Midland Provincial Park,https://www.google.com/maps/place/?q=place_id:...,2
...,...,...,...,...,...,...
449,50.759948,-96.542074,True,Elk Island Provincial Park,https://www.google.com/maps/place/?q=place_id:...,7
451,53.760861,-98.813876,False,Manitoba,https://www.google.com/maps/place/?q=place_id:...,7
453,51.044733,-114.071883,False,Calgary,https://www.google.com/maps/place/?q=place_id:...,8
458,46.248685,-60.851817,False,Cape Breton Island,https://www.google.com/maps/place/?q=place_id:...,11


In [15]:
# Specify the file path and name for the CSV file
file_path = 'park_info_file.csv'  # Replace with the desired file path and name

# Save the DataFrame to CSV
df_fn.to_csv(file_path)

In [16]:
df2 = df.loc[df.Ispark == False]
df2.head(20)

,Unnamed: 0,word,freq_count,Lat,Lon,Ispark,Full_name,URL
1,690,Moncton,1,46.087817,-64.778231,False,Moncton,https://www.google.com/maps/place/?q=place_id:...
3,680,Ireland,1,56.130366,-106.346771,False,Canada,https://www.google.com/maps/place/?q=place_id:...
6,659,Pemberton,1,50.322028,-122.805050,False,Pemberton,https://www.google.com/maps/place/?q=place_id:...
7,658,Whistler,1,50.116169,-122.953512,False,Whistler,https://www.google.com/maps/place/?q=place_id:...
8,656,Sea,1,56.130366,-106.346771,False,Canada,https://www.google.com/maps/place/?q=place_id:...
11,645,West Lion,1,49.457778,-123.186389,False,West Lion,https://www.google.com/maps/place/?q=place_id:...
14,641,Hope,1,56.130366,-106.346771,False,Canada,https://www.google.com/maps/place/?q=place_id:...
15,640,Needle Peak,1,49.571111,-121.125278,False,Needle Peak,https://www.google.com/maps/place/?q=place_id:...
17,624,Lake Superier Park,1,47.723087,-86.940716,False,Lake Superior,https://www.google.com/maps/place/?q=place_id:...
18,622,Huron Shelter,1,56.130366,-106.346771,False,Canada,https://www.google.com/maps/place/?q=place_id:...


In [17]:
df2.shape

(316, 8)